<img style="float: left;" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Chat Completion

Decision Intelligence Used in this module:
* Listing of various decision-making frameworks and what their descriptions  
* Recommendation of a decision-making framework for higher-stakes decisions (purchase of a car)  
* Illustrating a domain specific decision-making framework (millitary intelligence)  


In this module, the Semantic Kernel ability to create a chat experience will be introduced. This is a much richer experience than just sending simple prompts that are stateless and context is forgotten.

Semantic Kernel has first-class support for chat scenarios, where the user talks back and forth with the LLM, the arguments get populated with the history of the conversation. During each new run of the kernel, the arguments will be provided to the AI with content. This allows the LLM to know the historical context of the conversation.

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [2]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"

using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0

In [3]:
// Install the required NuGet packages
// Note: This also installs the Dependency Injection Package to retrieve the ChatCompletionService
#r "nuget: Microsoft.Extensions.DependencyInjection, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.13.0"

using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();


Installed Packages Microsoft.Extensions.DependencyInjection, 8.0.0 Microsoft.SemanticKernel, 1.13.0

Using Azure OpenAI Service


### Step 2 - Execute Chat Completion with Prompt Execution Settings

> If you can’t describe what you are doing as a process, you don’t know what you’re doing.   
>
> -- <cite>W. Edwards Deming (renowned American engineer, statistician, and management consultant)</cite>  


Using the Semantic Kernel ChatCompletion service is very similar to inovoking a prompt for basic LLM interactions. The chat completion service will provide very similar results to invoking the prompt directly.

In [4]:
// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 2500, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var simpleDecisionPrompt = """
Provide a list of decision frameworks that can help improve the quality of decisions.
""";
var chatResult = await chatCompletionService.GetChatMessageContentAsync(simpleDecisionPrompt, openAIPromptExecutionSettings);

Console.WriteLine(chatResult.Content);

Certainly! Here are several decision frameworks that can help improve the quality of decisions:

1. **SWOT Analysis**:
   - **Strengths**: Identify internal strengths.
   - **Weaknesses**: Identify internal weaknesses.
   - **Opportunities**: Identify external opportunities.
   - **Threats**: Identify external threats.

2. **PEST Analysis**:
   - **Political**: Consider political factors.
   - **Economic**: Consider economic factors.
   - **Social**: Consider social factors.
   - **Technological**: Consider technological factors.

3. **Decision Matrix (Weighted Scoring Model)**:
   - List options and criteria.
   - Assign weights to criteria based on importance.
   - Score each option against each criterion.
   - Calculate weighted scores to determine the best option.

4. **Cost-Benefit Analysis**:
   - Identify and quantify costs.
   - Identify and quantify benefits.
   - Compare costs and benefits to determine the best course of action.

5. **Pareto Analysis (80/20 Rule)**:
   - Iden

### Step 3 - Execute a Semantic Kernel Chat Completion with Streaming

Semantic Kernel supports prompt response streaming in the ChatCompletionService. This allows responses to be streamed to the client as soon as they are made available by the LLM and service. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

In [5]:
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(simpleDecisionPrompt))
{
    Console.Write(content);
}

Certainly! Here is a list of decision frameworks that can be highly useful in improving the quality of decisions in both personal and professional contexts:

1. **SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats)**
   - Evaluates internal and external factors to help understand the overall landscape before making a decision.

2. **PEST Analysis (Political, Economic, Social, Technological)**
   - Helps in understanding the macro-environmental factors that can affect an organization or decision.

3. **Decision Matrix (also known as a Prioritization Matrix)**
   - Uses a grid to compare different options based on multiple criteria and weights.

4. **Cost-Benefit Analysis**
   - Compares the benefits of a decision or action against its costs.

5. **Pareto Analysis (80/20 Rule)**
   - Focuses on identifying the most significant factors that will have the largest impact, helping prioritize efforts.

6. **Force Field Analysis**
   - Identifies and analyzes the forces for and again

### Step 4 - Execute a Semantic Kernel with Chat History

Semantic Kernel has a ChatHistory object that can be used with the ChatCompletionService to provide historical chat context to the LLM. Notice that the ChatHistory object differentiates between the different types of chat messages:
* System Message - System or MetaPrompt
* User Message - A message from the user
* Assistante Message - A message from the LLM

This is a more sophisticated approach than passing chat history in a long dynamic string. ChatHistory objects can be serialized and persisted into databases as well.

In [6]:
var systemPrompt = """
You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";
var simpleDecisionPrompt = "Provide five decision frameworks that can help improve the quality of decisions.";

var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(simpleDecisionPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);

Improving the quality of decisions can be crucial in both personal and professional contexts. Here are five widely-recognized decision frameworks that can help enhance the decision-making process:

### 1. **SWOT Analysis**

#### Overview:
SWOT Analysis is a straightforward framework used for strategic planning. It assesses strengths, weaknesses, opportunities, and threats.

#### How to Use:
- **Strengths:** Identify your internal advantages.
- **Weaknesses:** Recognize your internal disadvantages.
- **Opportunities:** Spot external factors that can be leveraged.
- **Threats:** Identify external challenges or risks.

#### Application:
SWOT is particularly useful for making strategic business decisions, project planning, and personal development.

### 2. **Pros and Cons List**

#### Overview:
A simple yet effective framework that lists the advantages and disadvantages of each option.

#### How to Use:
- Create two columns: one for Pros and one for Cons.
- List all the benefits and drawba

### Step 5 - Using Semantic Kernel Chat History in Conversation

With Semantic Kernel maintaining state with the ChatHistory object, a user can interact with the LLM in a natural way. Note the interaction below, where the user is asking which of the particular decision frameworks are best suited to purchase a car. Without the ChatHistory being passed along, the LLM would not have had the proper context to answer the question.

> Price is what you pay. Value is what you get.   
>
> -- <cite>Warren Buffet (one of the most successful investors of all time)</cite>  

In [11]:
var simpleDecisionPromptFollowup = """
Which of these 5 decision frameworks would work best for making a decision to purchase a car. 
Select only the single best one.
""";

// Add User message to the chat history
chatHistory.AddUserMessage(simpleDecisionPromptFollowup);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}

// Add Assistant message to the chat history
chatHistory.AddAssistantMessage(chatResponse);

For making a decision to purchase a car, the **Decision Matrix (Weighted Scoring Model)** is the single best framework.

### Why the Decision Matrix is Ideal for Purchasing a Car

#### Comprehensive Analysis
Purchasing a car is a complex decision that requires consideration of various factors such as:
- **Price**
- **Fuel Efficiency**
- **Safety**
- **Brand Reputation**
- **Comfort and Features**
- **Maintenance Costs**
- **Resale Value**

The Decision Matrix allows you to systematically assess and compare multiple car options based on these various criteria, ensuring a well-rounded decision.

### Steps to Use the Decision Matrix

1. **Identify Criteria**:
   - List all the factors that are important to your car purchase decision.

2. **Assign Weights**:
   - Determine the relative importance of each criterion by assigning a weight. For example, if safety is most important to you, it may be assigned a higher weight.

3. **Rate Each Car**:
   - Score each car option against each criteri

> Take time to deliberate, but when the time for action comes, stop thinking and go in.   
>
> -- <cite>Napoleon Bornaparte (French Emporer and brilliant military mind)</cite>  

In [13]:
var simpleDecisionPromptFollowupPartTwo = """
Which of these 5 decision frameworks listed above are used by the military intelligence community?
""";

// Add User message to the chat history
chatHistory.AddUserMessage(simpleDecisionPromptFollowupPartTwo);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}

// Add Assistant message to the chat history
chatHistory.AddAssistantMessage(chatResponse);

In the context of military and intelligence operations, two of the five decision frameworks listed—**SWOT Analysis** and the **OODA Loop (Observe, Orient, Decide, Act)**—are notably utilized. Here’s a detailed explanation of how they are applied:

### 1. OODA Loop

#### Overview
Developed by military strategist Colonel John Boyd, the **OODA Loop** is a strategic tool specifically designed for use in military operations. It facilitates rapid, adaptive decision-making in dynamic, high-stakes environments.

#### Steps
1. **Observe**: Gather information about the current situation, including intelligence data, environmental conditions, and enemy activities.
2. **Orient**: Analyze and synthesize the gathered information to build situational awareness. This involves understanding the context and potential implications.
3. **Decide**: Develop hypotheses and choose the most effective course of action based on the analyzed information.
4. **Act**: Implement the chosen action and continuously mo

### Step 6 - Semantic Kernel inspecting Chat History 

The Semantic Kernel Chat History is a transparent object that can be inspected and written out. In fact, the ChatHistory object can be manipulated to replay chats from middle interactions to simulate different outcomes.

Execute the cell below to write out the Semantic Kernel Chat History of this chat.

In [14]:
// Print the number of chat interactions and the chat history (turns)
Console.WriteLine("Number of chat interactions: " + chatHistory.Count());

foreach (var message in chatHistory)
{
    Console.WriteLine($"# {message.Role}: {message.Content}");
}

Number of chat interactions: 17
# system: You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
# user: Provide five decision frameworks that can help improve the quality of decisions.
# assistant: Improving the quality of decisions can be crucial in both personal and professional contexts. Here are five widely-recognized decision frameworks that can help enhance the decision-making process:

### 1. **SWOT Analysis**

#### Overview:
SWOT Analysis is a straightforward framework used for strategic planning. It assesses strengths, weaknesses, opportunities, and threats.

#### How to Use:
- **Strengths:** Identify your internal advantages.
- **Weaknesses:** Recognize your internal disadvantages.
- **Opportunities:** Spot external factors that can be leveraged.
- **Threats:** Identify external challenges or risk